In [ ]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

import pandas as pd
import numpy as np

from skimage.io import imread
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding



In [ ]:
img_width = 640
img_height = 480

PATH_TO_CSV = '/content/gdrive/My Drive/dare2compete/data/train.csv'
PATH = "/content/gdrive/My Drive/dare2compete/data/train/"

PATH_TO_MODEL = '/content/gdrive/My Drive/dare2compete/InceptionV3/'

In [ ]:
model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (img_height, img_width, 3))
#print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
80142336/80134624 [==============================] - 1s 0us/step


In [ ]:
for layer in model.layers[:]:
    layer.trainable = False

x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(4, activation="linear")(x)
print((model.layers))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
[<keras.engine.input_layer.InputLayer object at 0x7f89f8bd2588>, <keras.layers.convolutional.Conv2D object at 0x7f89a5752f98>, <keras.layers.convolutional.Conv2D object at 0x7f89a401ec50>, <keras.layers.pooling.MaxPooling2D object at 0x7f89a3fa57b8>, <keras.layers.convolutional.Conv2D object at 0x7f89a3fa58d0>, <keras.layers.convolutional.Conv2D object at 0x7f89a3fd9208>, <keras.layers.pooling.MaxPooling2D object at 0x7f89a3f726d8>, <keras.layers.convolutional.Conv2D object at 0x7f89a3f72668>, <keras.layers.convolutional.Conv2D object at 0x7f89a3f1f4e0>, <keras.layers.convolutional.Conv2D object at 0x7f89a3f3ba20>, <keras.layers.convolutional.Conv2D object at 0x7f89a3ee56d8>, <keras.layers.pooling.MaxPooling2D object at 0x7f89a3efea58>, <keras.layers.convolutional.Conv2D object at 0x7f89a3efecf8>, <keras.layers.convolutional.Conv2D object at 0x7f89a3eab9e8>, <keras.layers.

In [ ]:
model = Model(input = model.input, output = predictions)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  """Entry point for launching an IPython kernel.


In [ ]:
def get_image(NAME):
    image = imread(PATH+NAME)
    return image

In [3]:
def preprocess_image(image):
    mean=[0.485, 0.456, 0.406]  
    std=[0.229, 0.224, 0.225]
  
    image=image/255.0
    for i in range(3):
        image[:,:,i] = (image[:,:,i] - mean[i])/std[i]
    return image
  

In [4]:
def unpreprocess_image(image):
    mean=[0.485, 0.456, 0.406]  
    std=[0.229, 0.224, 0.225]
  
    for i in range(3):
        image[:,:,i] = (image[:,:,i] *std[i])+mean[i]
   
    image=image*255.0

    return image
  

In [ ]:
def image_generator(X, batch_size=16):
    while True:

        img_names = np.random.choice(X.index, size=batch_size)
        
        batch_x = []
        batch_y = [] 
          
        for img_name in img_names:
            img = get_image(img_name)
            y = X.loc[img_name].values
            #print(img.shape)

            img = preprocess_image(img)
            batch_x.append(img)
            batch_y.append(y)

        batch_x = np.array(batch_x)
        batch_y = np.array(batch_y)

        yield (batch_x, batch_y)

In [ ]:

def save_model(name,model):
    model_json = model.to_json()
    with open(PATH_TO_MODEL+ name+ ".json", "w") as json_file:
        json_file.write(model_json)
    model.save_weights(PATH_TO_MODEL+ name +".h5")
    print("Saved model to disk")


In [ ]:
def load_model(name):
    json_file = open(PATH_TO_MODEL+ name+ ".json", 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    loaded_model.load_weights(PATH_TO_MODEL+ name +".h5")
    print("Loaded model from disk")
    return loaded_model

In [ ]:
model.compile(loss = "mean_squared_error", optimizer = optimizers.Adam(lr=0.0028))


In [ ]:
import matplotlib.pyplot as plt
x = np.random.choice(X.index)
img = get_image(x)
box = X.loc[x].values

print(box[:2])
"""
plt.subplot(221)
plt.imshow(img)
print("original:",img)
img = preprocess_image(img)
print("processed: ",img)
plt.subplot(222)
plt.imshow(img)
img = unpreprocess_image(img)
print("unpreprocessed: ",img)
plt.subplot(223)
plt.imshow(img)
"""